<h1 id=\"tocheading\">Table of Contents</h1>
    <div id=\"toc\"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

// needed to generate the Table of contents 
// taken from github.com/kmahelona/ipython_notebook_goodies

<IPython.core.display.Javascript object>

# Modeling

## Fit Models with Training Data Set

## Review Model Outcomes — Iterate over additional models as needed.

## Identify the Final Model

In [13]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import MinMaxScaler

In [16]:
#Load data and scaler model

X_train_scaled = pd.read_csv("../data/X_train_scaled.csv").to_numpy()
X_test_scaled = pd.read_csv("../data/X_test_scaled.csv").to_numpy()

feature_list = pd.read_csv("../data/feature_list.csv")
feature_list = pd.Index(list(feature_list["0"]))

scaler = joblib.load("../data/scaler.pkl")

MinMaxScaler()

In [3]:
#initialize dict to collect model performance data
models_comparison = {'Model': [],
        'Explained Variance': [],
        'Mean Absolute Error': [],
        'Features Dropped': []
        }

In [19]:

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier


# from sklearn.naive_bayes import GaussianNB
# from sklearn.neural_network import MLPClassifier
# from sklearn.gaussian_process import GaussianProcessClassifier
# from sklearn.gaussian_process.kernels import RBF
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

h = .02  # step size in the mesh ???

names = ["Nearest Neighbors", "Linear SVM", "Decision Tree", "Random Forest", "AdaBoost",
         "Gradient Boosting"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GradientBoostingClassifier()]

all_params = { 'dictA': {'key_1': 'value_1'},
                'dictB': {'key_2': 'value_2'}}

6

In [31]:
from bayes_opt import BayesianOptimization
name = names[0]
model = classifiers[0]

params_boundaries = {'n_neighbors':(3,11)}

def kNN_hyper_param(k):
    params = {'n_neighbors': int(k),
              'n_job' : -1}
    clf = KNeighborsClassifier(params)
    scores = cross_val_score(clf, X, y, cv=5)
    
    return np.mean(cross_val_score(clf, X_train_scaled, y_train, cv=5, scoring='roc_auc'))

optimizer = BayesianOptimization(model, params)
optimizer.maximize(n_iter=10, init_points=2)

ValueError: setting an array element with a sequence.

In [24]:
for name, model in zip(names, classifiers):
    # instantiate classifier
    # already done outside
    
    #filter relevant params
    #params = all_params[name]
    #preform Bayesian Optimization
    optimizer = BayesianOptimization(model, params)
    

Linear SVM
AdaBoost
Gradient Boosting
